## Set up google colab environment

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification/V1/SGCNN_8')

In [6]:
from sample_extraction_V1_SGCNN_8 import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [7]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [8]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [9]:
data_source.shape

(145, 145, 200)

In [10]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [11]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [12]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [13]:
data_target.shape

(1476, 256, 145)

In [14]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [15]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [16]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [17]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [18]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [19]:
pretrain_source_models(training_set_size = [200],
                      classes = classes_source,
                      cube_size = 20,
                      overlap_ratio = 1,
                      data = data_source,
                      ground_truth = ground_truth_source,
                      batch_size = 20,
                      channels = 64,
                      epochs = 50,
                      Verbosity = 1,
                      accuracies = [],
                      learning_rate = 0.0001,
                      source_dataset = 'indian_pines')


Model training starts for data with 200 samples from each class in training set

Samples per class: [1368, 523, 323, 730, 356, 837, 2224, 460, 1085]
Total number of samples 7906.

Total number of samples in training set 1800.
unique classes in training set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in training set: [200 200 200 200 200 200 200 200 200]

Total number of samples in test set 6106.
unique classes in test set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in test set: [1168  323  123  530  156  637 2024  260  885]

X_train => (1800, 20, 20, 64)
X_test  => (6106, 20, 20, 64)
y_train => (1800, 9)
y_test  => (6106, 9)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
_____________________________________________

# Fine tune on botswana

In [20]:
transfer_results, confusion_matrixes = transfer_learning(source_dataset = 'indian_pines',
                                        target_dataset = 'botswana',
                                        data = data_target,
                                        ground_truth = ground_truth_target,
                                        training_samples_from_each_class = [15,30,45,60,75],
                                        source_training_size = [200],
                                        classes = classes_target,
                                        overlap_ratio = 1,
                                        channels = 64,
                                        cube_size = 20,
                                        learning_rate = 0.0001,
                                        epochs = 150,
                                        batch_size = 20,
                                        test_accuracies = [],
                                        )


Model training starts for data with 15 samples from each class in training set

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv_initial (Conv2D)           (None, 10, 10, 64)   36928       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 10, 10, 64)   256         conv_initial[0][0]               
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 10, 10, 64)   0           batch_normalization[0][0]        
___________

# Transfer Learning results on Botswana

In [21]:
transfer_results

,Training samples per class finetuning,Training Samples per class pretraining,Training Samples,Test Samples,Test_Accuracies
0,15,200,210,2934,66.84
1,30,200,420,2724,66.84
2,45,200,630,2514,70.67
3,60,200,840,2304,70.67
4,75,200,1050,2094,71.92


# Classification accuracies per class for each model

In [22]:
confusion_matrixes[0]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,170,0,0,0,61,0,24,0,0,0,0,0,0,0,66.67
1,0,85,0,1,0,0,0,0,0,0,0,0,0,0,98.84
2,0,0,60,148,9,0,5,5,9,0,0,0,0,0,25.42
3,0,15,1,183,0,0,0,0,0,0,0,0,1,0,91.5
4,16,16,0,22,107,11,32,17,1,19,5,0,8,0,42.13
5,0,18,0,7,0,48,0,0,44,0,0,1,136,0,18.9
6,28,0,0,0,0,0,216,0,0,0,0,0,0,0,88.52
7,0,0,0,0,0,0,0,98,0,0,13,64,13,0,52.13
8,0,0,0,0,0,0,0,3,250,10,0,0,36,0,83.61
9,0,0,0,0,0,0,0,16,0,185,29,0,3,0,79.4


In [23]:
confusion_matrixes[1]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,155,0,0,0,74,0,11,0,0,0,0,0,0,0,64.58
1,0,71,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,120,78,13,0,1,5,4,0,0,0,0,0,54.3
3,0,12,3,159,0,0,0,0,0,0,0,0,11,0,85.95
4,21,15,1,21,104,8,17,42,0,1,1,0,8,0,43.51
5,0,14,0,7,0,35,0,0,43,0,0,5,135,0,14.64
6,26,0,0,0,0,0,203,0,0,0,0,0,0,0,88.65
7,0,0,0,0,0,0,0,140,0,0,3,28,2,0,80.92
8,0,0,0,4,0,0,0,12,230,3,0,0,35,0,80.99
9,0,0,0,0,0,0,0,12,0,199,7,0,0,0,91.28


In [24]:
confusion_matrixes[2]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,157,0,0,0,65,0,3,0,0,0,0,0,0,0,69.78
1,0,56,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,164,21,13,0,1,1,2,4,0,0,0,0,79.61
3,0,9,6,114,0,0,0,0,0,0,0,0,41,0,67.06
4,23,10,1,19,91,13,14,42,0,2,0,0,9,0,40.62
5,0,14,0,0,0,26,0,1,51,0,0,2,130,0,11.61
6,22,0,0,0,0,0,192,0,0,0,0,0,0,0,89.72
7,0,0,0,0,0,0,0,136,0,0,2,19,1,0,86.08
8,0,16,0,0,0,0,0,0,211,17,0,0,25,0,78.44
9,0,0,0,0,0,0,0,0,0,201,2,0,0,0,99.01


In [25]:
confusion_matrixes[3]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,140,0,0,0,70,0,0,0,0,0,0,0,0,0,66.67
1,0,41,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,141,29,12,0,2,1,3,3,0,0,0,0,73.82
3,0,0,1,96,0,0,0,0,0,0,0,0,58,0,61.94
4,18,4,0,26,107,14,0,33,0,0,0,0,7,0,51.2
5,0,13,0,0,0,15,0,2,51,0,0,1,127,0,7.18
6,10,0,0,0,0,0,189,0,0,0,0,0,0,0,94.97
7,0,0,0,0,0,0,0,126,0,0,2,15,0,0,88.11
8,0,13,0,0,0,0,0,4,214,12,0,0,11,0,84.25
9,0,0,0,0,0,0,0,0,0,183,5,0,0,0,97.34


In [26]:
confusion_matrixes[4]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,172,0,0,0,23,0,0,0,0,0,0,0,0,0,88.21
1,0,26,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,95,45,0,0,1,0,25,10,0,0,0,0,53.98
3,0,0,0,70,0,0,0,0,0,0,0,0,70,0,50.0
4,17,5,5,12,103,16,0,23,1,0,0,0,12,0,53.09
5,0,9,0,0,0,18,0,2,37,0,0,0,128,0,9.28
6,0,0,0,0,0,0,184,0,0,0,0,0,0,0,100.0
7,0,0,0,0,0,0,0,127,0,0,0,0,1,0,99.22
8,0,16,0,0,0,0,0,0,199,17,0,0,7,0,83.26
9,0,0,0,0,0,0,0,0,0,169,4,0,0,0,97.69
